## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
import os
sys.path.append(os.path.abspath('..'))
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Chokurdakh,RU,70.6333,147.9167,-19.00,94,98,3.17,Overcast Clouds,2021-11-11 06:05:41
1,1,Cabo San Lucas,MX,22.8909,-109.9124,25.28,71,0,1.33,Clear Sky,2021-11-11 06:02:39
2,2,Praia,CV,14.9215,-23.5087,23.30,83,0,3.09,Clear Sky,2021-11-11 06:05:42
3,3,Punta Arenas,CL,-53.1500,-70.9167,8.95,49,53,10.15,Broken Clouds,2021-11-11 06:01:03
4,4,Ushuaia,AR,-54.8000,-68.3000,4.81,93,0,1.03,Clear Sky,2021-11-11 06:05:43


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 15
What is the maximum temperature you would like for your trip? 23


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
6,6,Bluff,NZ,-46.6000,168.3333,15.75,78,96,6.95,Overcast Clouds,2021-11-11 06:01:12
20,20,Mar Del Plata,AR,-38.0023,-57.5575,16.16,82,20,0.45,Few Clouds,2021-11-11 06:05:51
21,21,Port Alfred,ZA,-33.5906,26.8910,19.37,71,34,5.55,Scattered Clouds,2021-11-11 06:05:52
23,23,Saldanha,ZA,-33.0117,17.9442,17.64,83,97,5.63,Overcast Clouds,2021-11-11 06:05:53
26,26,Oranjemund,NaN,-28.5500,16.4333,17.51,69,1,5.28,Clear Sky,2021-11-11 06:05:54
27,27,Bredasdorp,ZA,-34.5322,20.0403,19.60,69,75,2.47,Broken Clouds,2021-11-11 06:05:55
30,30,Tateyama,JP,34.9833,139.8667,21.68,54,0,12.65,Clear Sky,2021-11-11 06:05:56
32,32,Tshikapa,CD,-6.4167,20.8000,22.73,87,100,0.15,Overcast Clouds,2021-11-11 06:05:57
34,34,Hermanus,ZA,-34.4187,19.2345,19.89,71,74,3.71,Broken Clouds,2021-11-11 06:05:58
39,39,Cape Town,ZA,-33.9258,18.4232,20.61,68,68,1.79,Broken Clouds,2021-11-11 06:02:30


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Bluff,NZ,15.75,Overcast Clouds,-46.6000,168.3333,
20,Mar Del Plata,AR,16.16,Few Clouds,-38.0023,-57.5575,
21,Port Alfred,ZA,19.37,Scattered Clouds,-33.5906,26.8910,
23,Saldanha,ZA,17.64,Overcast Clouds,-33.0117,17.9442,
27,Bredasdorp,ZA,19.60,Broken Clouds,-34.5322,20.0403,
30,Tateyama,JP,21.68,Clear Sky,34.9833,139.8667,
32,Tshikapa,CD,22.73,Overcast Clouds,-6.4167,20.8000,
34,Hermanus,ZA,19.89,Broken Clouds,-34.4187,19.2345,
39,Cape Town,ZA,20.61,Broken Clouds,-33.9258,18.4232,
43,Puerto Ayora,EC,20.75,Scattered Clouds,-0.7393,-90.3518,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in {row['City']}... skipping.")

Hotel not found in Adrar... skipping.
Hotel not found in Tombouctou... skipping.
Hotel not found in Marzuq... skipping.
Hotel not found in Tall Kayf... skipping.
Hotel not found in Mitu... skipping.
Hotel not found in Taoudenni... skipping.
Hotel not found in Kunya... skipping.
Hotel not found in Amapa... skipping.
Hotel not found in Boende... skipping.
Hotel not found in Araouane... skipping.
Hotel not found in Kembe... skipping.
Hotel not found in Baoro... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

In [10]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Bluff,NZ,15.75,Overcast Clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
20,Mar Del Plata,AR,16.16,Few Clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
21,Port Alfred,ZA,19.37,Scattered Clouds,-33.5906,26.8910,The Halyards Hotel
23,Saldanha,ZA,17.64,Overcast Clouds,-33.0117,17.9442,Blue Bay Lodge
27,Bredasdorp,ZA,19.60,Broken Clouds,-34.5322,20.0403,Bredasdorp Country Manor
...,...,...,...,...,...,...,...
689,San Jose De Guanipa,VE,22.28,Broken Clouds,8.8872,-64.1651,HOTEL 283 SUITES
690,Muroto,JP,17.02,Few Clouds,33.2833,134.1500,Oota Ryokan
694,Puerto Leguizamo,CO,22.06,Light Rain,-0.1934,-74.7819,la casona de juancho
695,Saquarema,BR,20.81,Overcast Clouds,-22.9200,-42.5103,Hotel Fazenda Sol e Lua


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and a Maximum Temperature of {Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))